In [ ]:
import duckdb

print(duckdb.__version__)
con = duckdb.connect(database=":memory:")

In [ ]:
duckdb.install_extension("icu")
duckdb.load_extension("icu")

con.execute("""
SELECT extension_name, installed, description
FROM duckdb_extensions();
            """).fetchdf()

In [ ]:
con.execute("""
SELECT name, abbrev
FROM pg_timezone_names()
ORDER BY name;
""").fetchdf()


In [ ]:
# Load Games

from pathlib import Path
from duckdb import DuckDBPyConnection


def ensure_user_view(games_path: Path, tz: str = "America/Los_Angeles"
) -> tuple[DuckDBPyConnection, str]:
    con.execute(f"SET TimeZone = '{tz}';")

    view = f"games"
    con.execute(
        f"""
        CREATE OR REPLACE VIEW {view} AS
        SELECT
            *
        FROM parquet_scan('{games_path.as_posix()}');
    """
    )
    return con, view

ensure_user_view(Path('.cache/joeshift/games.parquet'))

In [ ]:
con.execute("DESCRIBE SELECT * FROM games;").df()

In [ ]:
con.execute("SELECT * from games").fetchdf()

In [ ]:
# fetch min/max timestamps
import pandas as pd


min_max = con.execute("""
    SELECT
      date_trunc('month', MIN(end_time)) AS min_month,
      date_trunc('month', MAX(end_time)) AS max_month
    FROM games;
""").fetchone()

if min_max is None:
    raise ValueError("Cannot get min/max from end_time")
min_m, max_m = pd.to_datetime(min_max[0]), pd.to_datetime(min_max[1])

# build continuous month sequence
months = pd.period_range(start=min_m, end=max_m, freq="M")
labels = months.strftime("%Y-%m").tolist()

labels

In [ ]:
lst = [r[0] for r in con.execute(
    "SELECT DISTINCT rated FROM games ORDER BY rated;"
).fetchall()]
lst

In [ ]:
lst = [r[0] for r in con.execute(
    "SELECT DISTINCT time_class FROM games;"
).fetchall()]

In [ ]:
from typing import Any

from utils.session import CurrentFilters


SQL = """
WITH filtered AS (
  SELECT *
  FROM games
  WHERE 1=1
    {w_time}
    {w_class}
    {w_rated}
),
base AS (
  SELECT
    COUNT(*)                            AS total,
    AVG(opponent_rating)                AS avg_opponent_rating,
    SUM(user_result_simple = 'win')     AS wins,
    SUM(user_result_simple = 'draw')    AS draws,
    SUM(user_result_simple = 'loss')    AS losses
  FROM filtered
),
rated AS (
  SELECT
    COUNT(*)                                AS n_rated,
    arg_min(user_rating, end_time)    AS first_r,
    arg_max(user_rating, end_time)    AS last_r
  FROM filtered
  WHERE rated = TRUE
)
SELECT
  base.total,
  COALESCE(wins::DOUBLE  / NULLIF(base.total, 0), 0) AS win_rate,
  COALESCE(draws::DOUBLE / NULLIF(base.total, 0), 0) AS draw_rate,
  COALESCE(losses::DOUBLE/ NULLIF(base.total, 0), 0) AS loss_rate,
  base.avg_opponent_rating,
  CASE WHEN rated.n_rated >= 2 THEN last_r - first_r ELSE 0 END AS rated_delta
FROM base CROSS JOIN rated;
"""

w_time, w_class, w_rated = "", "", ""
params: list[Any] = []
f = CurrentFilters(
    month_start="2025-09",
    month_end="2025-10",
    time_class="All",
    rated_only=True
)
start = pd.Period(f.month_start, freq="M").to_timestamp("M") if f.month_start else None
end = (pd.Period(f.month_end, freq="M") + 1).to_timestamp("M") if f.month_end else None
if start is not None:
    w_time += " AND end_time >= ?"
    params.append(pd.Timestamp(start).to_pydatetime())
if end is not None:
    w_time += " AND end_time < ?"
    params.append(pd.Timestamp(end).to_pydatetime())

if f.time_class and f.time_class != "All":
    w_class += " AND time_class = ?"
    params.append(f.time_class)

if f.rated_only is True:
    w_rated += " AND rated = TRUE"
elif f.rated_only is False:
    w_rated += " AND rated = FALSE"

q = SQL.format(w_time=w_time, w_class=w_class, w_rated=w_rated)
cur = con.execute(q, params)  # used in both CTEs by position

row = cur.fetchone()
if not row:
    return Summary(total=0, win_rate=0.0, draw_rate=0.0, loss_rate=0.0,
                    avg_opponent_rating=None, rated_delta=0)
cols = [d[0] for d in cur.description]
return Summary.model_validate(dict(zip(cols, row)))